In [3]:
import pandas as pd
import numpy as np

In [4]:
load_stuff=False

replay_capacity=2**20 #needs to be a power of 2 for PERB implementation
batch_size=256
start_alpha=0.6 #see section B.2.2 (pg. 14 table 3) in paper: https://arxiv.org/pdf/1511.05952.pdf
replay_epslion=0.01 #introduced on page 4 in paper: https://arxiv.org/pdf/1511.05952.pdf
beta=0.4 #needs to satisfy 0<=beta<=1; annealed linearly to 1; see page 5 of paper: https://arxiv.org/pdf/1511.05952.pdf


seed_braids=[[1, 1, 1],
             [1, -2, 1, -2],
             [1, 1, 1, 2, -1, 2],
             [1, 1, -2, 1, -2, -2],
             [1, 1, 1, -2, 1, -2]]
start_states_capacity=1000000
max_braid_index=6
max_braid_length=10

uniform=True
move_penalties=[0.07, 0.09, 0.1, 0.15]
seed_probs=[0.1, 0.5]

#try longer networks
#Double Dueling DQN
output_size=13 #should be the number of actions the agent can take in the MDP
architectures_list = [{'Hidden': (512, 512, 512), 'Value': (512, 1),'Advantage': (512, 13)},
                      {'Hidden': (1024, 1024, 1024), 'Value': (1024, 1),'Advantage': (1024, 13)},
                      {'Hidden': (2048, 2048, 2048), 'Value': (2048, 1),'Advantage': (2048, 13)}]
                      

transfer_rate=2000
gamma=0.99
learning_rates=[0.000000001, 0.0000000003, 0.000000001]

#Training
euler_char_reset=-8 #algorithm will initialize state if any eulerchar falls below euler_char_reset
max_actions_length=40 #initialize_state() is called if an episode takes more actions than max_actions_length

#epsilon parameters to linearly decrease epsilon from start_epsilon to final_epsilon over
#num_decrease_epochs. If a model is loaded (i.e. load_stuff=True), epsilon wil be the
#final_epsilon and will not change.
start_epsilon=1
final_epsilon=0.1
num_decrease_epochs=250000

store_rate=10000 #how often (in epochs) to store values for matplotlib lists
report_policy_rate=1000
num_epochs=2000000
moves_per_epoch=4

In [9]:
columns=["load_stuff", "load_job_name", "save_job_name", "replay_capacity", "batch_size", "start_alpha", "batch_size", 
         "repaly_epsilon", "beta", "seed_braids", "start_states_capacity", "max_braid_index", "max_braid_length", "uniform", 
         "move_penalty", "seed_prob", "output_size", "architextures", "transfer_rate", "gamma", "learning_rate", 
         "euler_char_reset", "max_actions_length", "start_epsilon", "final_epsilon", "num_decrease_epochs", "store_rate",
         "report_policy_rate", "num_epochs", "moves_per_epoch"]
load_stuff=False
job_name="SliceEnv_try_"
i=0
df = pd.DataFrame(columns=columns)
for move_penalty in move_penalties:
    for seed_prob in seed_probs:
        for architextures in architectures_list:
            for learning_rate in learning_rates:
                vals=[load_stuff,
                      job_name+str(i),
                      job_name+str(i),
                      replay_capacity,
                      batch_size,
                      start_alpha,
                      replay_epslion,
                      beta,
                      seed_braids,
                      start_states_capacity,
                      max_braid_index,
                      max_braid_length,
                      uniform,
                      move_penalty,
                      seed_prob,
                      output_size,
                      architextures,
                      transfer_rate,
                      gamma,
                      learning_rate,
                      euler_char_reset,
                      max_actions_length,
                      start_epsilon,
                      final_epsilon,
                      num_decrease_epochs,
                      store_rate,
                      report_policy_rate,
                      num_epochs,
                      moves_per_epoch]
                row=dict(zip(columns, vals))
                df=df.append(row, ignore_index=True)
                i+=1                   


NameError: name 'start_alpha' is not defined

In [6]:
df.to_msgpack(path_or_buf="hyperparameter_df")

In [7]:
df2=pd.read_msgpack(path_or_buf="hyperparameter_df")

In [8]:
df2.head()

,load_stuff,load_job_name,save_job_name,replay_capacity,batch_size,seed_braids,start_states_capacity,max_braid_index,max_braid_length,uniform,...,learning_rate,euler_char_reset,max_actions_length,start_epsilon,final_epsilon,num_decrease_epochs,store_rate,report_policy_rate,num_epochs,moves_per_epoch
0,False,SliceEnv_try_0,SliceEnv_try_0,1000000.0,1024.0,"((1, 1, 1), (1, -2, 1, -2), (1, 1, 1, 2, -1, 2...",1000000.0,6.0,10.0,True,...,1.000000e-09,-8.0,40.0,1.0,0.1,250000.0,10000.0,1000.0,2000000.0,4.0
1,False,SliceEnv_try_1,SliceEnv_try_1,1000000.0,1024.0,"((1, 1, 1), (1, -2, 1, -2), (1, 1, 1, 2, -1, 2...",1000000.0,6.0,10.0,True,...,3.000000e-10,-8.0,40.0,1.0,0.1,250000.0,10000.0,1000.0,2000000.0,4.0
2,False,SliceEnv_try_2,SliceEnv_try_2,1000000.0,1024.0,"((1, 1, 1), (1, -2, 1, -2), (1, 1, 1, 2, -1, 2...",1000000.0,6.0,10.0,True,...,1.000000e-09,-8.0,40.0,1.0,0.1,250000.0,10000.0,1000.0,2000000.0,4.0
3,False,SliceEnv_try_3,SliceEnv_try_3,1000000.0,1024.0,"((1, 1, 1), (1, -2, 1, -2), (1, 1, 1, 2, -1, 2...",1000000.0,6.0,10.0,True,...,1.000000e-09,-8.0,40.0,1.0,0.1,250000.0,10000.0,1000.0,2000000.0,4.0
4,False,SliceEnv_try_4,SliceEnv_try_4,1000000.0,1024.0,"((1, 1, 1), (1, -2, 1, -2), (1, 1, 1, 2, -1, 2...",1000000.0,6.0,10.0,True,...,3.000000e-10,-8.0,40.0,1.0,0.1,250000.0,10000.0,1000.0,2000000.0,4.0


In [ ]:
df2["move_penalty"]

In [ ]:
2**17